## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Optimizer, AdamW
from torch.optim.lr_scheduler import LambdaLR, CyclicLR, OneCycleLR
from sklearn.metrics import f1_score

# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 32.8 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## 하이퍼파라미터

In [ ]:
EPOCHS = 50
LR = 1e-2
BS = 16384 # 2의 제곱승꼴
SEED = 41



## 시드고정

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

## 데이터로드

In [ ]:
train = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/train.csv')
train = train.drop(columns=['ID'])
val = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/val.csv')
val = val.drop(columns=['ID'])
test = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/test.csv')
test = test.drop(columns=['ID'])

print(train.shape, val.shape, test.shape)

(113842, 30) (28462, 31) (142503, 30)


## 데이터셋 생성

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            x = torch.from_numpy(self.df[index]).type(torch.FloatTensor)
            y = torch.FloatTensor([self.labels[index]])
            return x, y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [ ]:
train_dataset = MyDataset(df=train, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=6)

val_dataset = MyDataset(df = val, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=6)

## 1D AutoEncoder

In [ ]:
# 계층 정규화
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

        self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon) # 계층정규화 완료
        return self.weight * x + self.bias # wx+b
        
# 활성화 함수
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        
class AutoEncoder1(nn.Module):
    def __init__(self):
        super(AutoEncoder1, self).__init__()
        
        
        self.ln = LayerNorm(5000)
        self.ln1 = LayerNorm(3500)
        self.ln2 = LayerNorm(2000)
        self.ln3 = LayerNorm(1000)
        
        self.upblock1 = nn.Sequential(nn.Linear(30, 1000), nn.BatchNorm1d(1000),GELU())
        self.upblock2 = nn.Sequential(nn.Linear(1000,2000), nn.BatchNorm1d(2000),GELU())
        self.upblock3 = nn.Sequential(nn.Linear(2000,3500), nn.BatchNorm1d(3500),GELU())
        self.upblock4 = nn.Sequential(nn.Linear(3500,5000), nn.BatchNorm1d(5000),GELU())

        self.downblock1 = nn.Sequential(nn.Linear(5000, 3500),nn.BatchNorm1d(3500),GELU())
        self.downblock2 = nn.Sequential(nn.Linear(3500, 2000),nn.BatchNorm1d(2000),GELU())
        self.downblock3 = nn.Sequential(nn.Linear(2000, 1000),nn.BatchNorm1d(1000),GELU())
        self.downblock4 = nn.Sequential(nn.Linear(1000, 300),nn.BatchNorm1d(300),GELU())
        
        self.fclayer = nn.Sequential(nn.Linear(300,30))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        upblock1_out = self.upblock1(x) 
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)
        
        downblock1_out = self.downblock1(self.ln(upblock4_out)) 
        skipblock1 = downblock1_out + upblock3_out 
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out 
        downblock4_out = self.downblock4(self.ln3(skipblock3))
        
        x = self.fclayer(downblock4_out)
         
        return x # 4

## Train (학습)

In [ ]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self):
        self.model.to(self.device)
        best_score = 0
        avg = 1
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                self.optimizer.zero_grad()

                _x = self.model(x)
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 0.95)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score <= score and avg > np.mean(train_loss):
                best_score = score
                avg = np.mean(train_loss)
                torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
                print('---------------------------')
                print(f'Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')
    
    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)

                _x = self.model(x)
                diff = cos(x, _x).cpu().tolist()
                batch_pred = np.where(np.array(diff)<thr, 1, 0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

## 모델 학습

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

model = nn.DataParallel(AutoEncoder1())
model.eval()

#optimizer = AdamW(params = model.parameters(), lr = LR, eps=1e-06)
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.00005, step_size_up=5, max_lr=0.001, gamma=0.5, mode='exp_range', cycle_momentum=False)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.5130304609026227] Val Score : [0.01770493016168002])
---------------------------
Train loss : [0.5130304609026227] Val Score : [0.01770493016168002])
Epoch : [1] Train loss : [0.24234039655753545] Val Score : [0.2820413080318538])
---------------------------
Train loss : [0.24234039655753545] Val Score : [0.2820413080318538])
Epoch : [2] Train loss : [0.1678934863635472] Val Score : [0.4936036828766804])
---------------------------
Train loss : [0.1678934863635472] Val Score : [0.4936036828766804])
Epoch : [3] Train loss : [0.1274620837398938] Val Score : [0.5078511110326694])
---------------------------
Train loss : [0.1274620837398938] Val Score : [0.5078511110326694])
Epoch : [4] Train loss : [0.10653261521032878] Val Score : [0.5133823628543464])
---------------------------
Train loss : [0.10653261521032878] Val Score : [0.5133823628543464])
Epoch : [5] Train loss : [0.09249477088451385] Val Score : [0.5239044751074692])
---------------------------
Train

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1639 MB |   11821 MB |  154233 GB |  154232 GB |\n|       from large pool |    1635 MB |   11816 MB |  154202 GB |  154200 GB |\n|       from small pool |       3 MB |       5 MB |      31 GB |      31 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1639 MB |   11821 MB |  154233 GB |  154232 GB |\n|       from large pool |    1635 MB |   11816 MB |

## 추론

In [ ]:
model = AutoEncoder1()
model.load_state_dict(torch.load('./best_model.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder1(
    (ln): LayerNorm()
    (ln1): LayerNorm()
    (ln2): LayerNorm()
    (ln3): LayerNorm()
    (upblock1): Sequential(
      (0): Linear(in_features=30, out_features=1000, bias=True)
      (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (upblock2): Sequential(
      (0): Linear(in_features=1000, out_features=2000, bias=True)
      (1): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (upblock3): Sequential(
      (0): Linear(in_features=2000, out_features=3500, bias=True)
      (1): BatchNorm1d(3500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (upblock4): Sequential(
      (0): Linear(in_features=3500, out_features=5000, bias=True)
      (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (downblock1): Sequentia

In [ ]:
test = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/test.csv')
test = test.drop(columns=['ID'])

In [ ]:
test_dataset = MyDataset(test, False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=6)

In [ ]:
def prediction(model, thr, test_loader, device):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            _x = model(x)
            
            diff = cos(x, _x).cpu().tolist()
            batch_pred = np.where(np.array(diff)<thr, 1,0).tolist()
            pred += batch_pred
    return pred

In [ ]:
preds = prediction(model, 0.95, test_loader, device)

In [ ]:
submit = pd.read_csv('./drive/MyDrive/input/신용카드 사기 거래 탐지/sample_submission.csv')
submit['Class'] = preds
submit.to_csv('./submit_autoencoder2.csv', index=False)